In [1]:
# Process raw SDWA csv files to county-level metrics for drinking water treatment violations dashboard
# A Grimm last updated 4/17/2023
# conda environment for script: N:\Projects\Blue Accounting\Data Management\DrinkingWater\Treatment\environment.yml
# compared to version 3 this version removes LCR violations (using output from 'Removing SDWA LCR violations.ipynb' and inserts tribe names

############################################
import pandas as pd
from functools import reduce
from sklearn.preprocessing import LabelEncoder as le
import os, csv, glob

# setup:
# go to https://echo.epa.gov/trends/comparative-maps-dashboards/drinking-water-dashboard
# for the filters, select the 8 GL states and for water system type, select 'community'
# In the 'Violations' widget, select 'Details' from the dropdown to generate table and click download; save to treatment folder
# Similarly, download 'Details' from the 'Public Water Systems (PWSs)' widget and save to systems folder
# This will exclude tribal systems, so repeat the above downloads but filter 'Primary Agency Type' to EPA and select EPA Regions 02, 03, and 05 and 'community' under water system type to download all tribal systems that may be in GL states
# download SDWA dataset from https://echo.epa.gov/tools/data-downloads, save at least SDWA_PUB_WATER_SYSTEMS.csv to same folder

# file paths to adjust as necessary:
# folders containing state and tribal downloads from drinking water dashboard - see above notes
tfldr = r"N:\Projects\Blue Accounting\Issues\Drinking Water\Water treatment\SDWA_dashboard_downloads_2023\treatment"
sfldr = r"N:\Projects\Blue Accounting\Issues\Drinking Water\Water treatment\SDWA_dashboard_downloads_2023\systems"
# list of PWSs from SW database (systems inside Great Lakes Basin)
pwstbl = pd.read_csv(r"N:\Projects\Blue Accounting\Issues\Drinking Water\Water treatment\SWpoints_Basin_All_Apr2022_M.csv") # https://glcommission.maps.arcgis.com/home/item.html?id=4f3b9a22a2d74f09896b6443479b5870
lcrtbl = pd.read_csv(r"N:\Projects\Blue Accounting\Issues\Drinking Water\Water treatment\GLPWSs_LCR.csv")

# range of years to summarize data for
startyear = 2012
endyear = 2022



In [5]:
# merge together the state and tribal water systems downloads from dashboard
'''
def read_sheets(filename):
    result = []
    sheets = pd.read_excel(filename, sheet_name=None)
    for name, sheet in sheets.items():
        sheet['Sheetname'] = name
        sheet['Row'] = sheet.index
        result.append(sheet)
    return pd.concat(result, ignore_index=True)

def read_files(filenames):
    result = []
    for filename in filenames:
        file = read_sheets(filename)
        file['Filename'] = filename
        result.append(file)
    return pd.concat(result, ignore_index=True)
'''

filenames = glob.glob(tfldr + "\*.xlsx")
print('File names:', filenames)

#tbl = pd.DataFrame()
tbl = list()

for file in filenames:
    df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
    tbl.append(df)
tbl = pd.concat(tbl, ignore_index=True)
    

# fix column names
tbl.columns = [c.lower().replace(' ', '_') for c in tbl.columns]

File names: ['N:\\Projects\\Blue Accounting\\Issues\\Drinking Water\\Water treatment\\SDWA_dashboard_downloads_2023\\treatment\\34370b01-035b-4800-bf05-61d83c6d3803.xlsx', 'N:\\Projects\\Blue Accounting\\Issues\\Drinking Water\\Water treatment\\SDWA_dashboard_downloads_2023\\treatment\\6f3c8c0e-e44d-4cbe-a8e1-0ab6381e7fe0.xlsx']


In [9]:
##formatting of pwstbl

#fix stripped leading zeroes
pwstbl['PWSID'] = pwstbl['PWSID'].str.rjust(9,'0')

# only keep US PWS's from source water dashboard for treatment map
pwstbl = pwstbl.loc[(pwstbl['Country'] == 'USA')]

# list of PWS's to include
PWSs = pwstbl['PWSID'].tolist()
PWSs = [x for x in PWSs if x != 'nan']

# print(sorted(PWSs))

In [10]:
# create subset of original data to match systems included in source water dashboard
GLPWSs = tbl[(tbl['pws_id'].isin(PWSs))]
print(len(GLPWSs.index))


systems = GLPWSs["pws_id"].unique() 
systems.sort()


9994


In [11]:
#how many systems from SDWIS violations data
print(len(systems))
#how many systems total in region from SW dashboard - should be larger than first number
print(len(PWSs))

2422
3210


In [12]:
GLPWSs.head()

,pws_id,pws_name,detailed_facility_report,calendar_year,primacy_agency_type,state/territory/tribe,city_served,population_served,water_system_type,system_size,primary_source_water,violation_type,violations
14,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2015,EPA,Unknown Tribe,-,5500,Community,Medium,Surface Water (SW),Monitoring & Reporting,1
15,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2016,EPA,St. Regis Band of Mohawk Indians of New York,-,5500,Community,Medium,Surface Water (SW),Monitoring & Reporting,1
16,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2020,EPA,St. Regis Band of Mohawk Indians of New York,-,5500,Community,Medium,Surface Water (SW),Monitoring & Reporting,2
17,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2021,EPA,St. Regis Band of Mohawk Indians of New York,-,5500,Community,Medium,Surface Water (SW),Acute Health-Based,1
18,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2021,EPA,St. Regis Band of Mohawk Indians of New York,-,5500,Community,Medium,Surface Water (SW),Monitoring & Reporting,1


In [13]:
# add empty columns for violation types
GLPWSs['HB'] = 0
GLPWSs['AHB'] = 0
GLPWSs['M_R'] = 0
GLPWSs['PN_O'] = 0
#populate columns
GLPWSs.loc[GLPWSs['violation_type'].eq('Health-Based'),'HB']=GLPWSs.loc[GLPWSs['violation_type'].eq('Health-Based'),'violations']
GLPWSs.loc[GLPWSs['violation_type'].eq('Acute Health-Based'),'AHB']=GLPWSs.loc[GLPWSs['violation_type'].eq('Acute Health-Based'),'violations']
GLPWSs.loc[GLPWSs['violation_type'].eq('Monitoring & Reporting'),'M_R']=GLPWSs.loc[GLPWSs['violation_type'].eq('Monitoring & Reporting'),'violations']
GLPWSs.loc[GLPWSs['violation_type'].eq('Public Notification and Other'),'PN_O']=GLPWSs.loc[GLPWSs['violation_type'].eq('Public Notification and Other'),'violations']

C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\2996194335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GLPWSs['HB'] = 0
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\2996194335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GLPWSs['AHB'] = 0
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\2996194335.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [15]:
GLPWSs_summ = GLPWSs.groupby(['pws_id','calendar_year']).sum().reset_index()

C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\1293096341.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  GLPWSs_summ = GLPWSs.groupby(['pws_id','calendar_year']).sum().reset_index()


In [16]:
GLPWSs_summ = GLPWSs_summ.drop('violations', axis=1)
GLPWSs_summ['Total'] = GLPWSs_summ['HB'] + GLPWSs_summ['AHB'] + GLPWSs_summ['M_R'] + GLPWSs_summ['PN_O']
GLPWSs_summ.head()

,pws_id,calendar_year,population_served,HB,AHB,M_R,PN_O,Total
0,020000005,CY 2015,5500,0,0,1,0,1
1,020000005,CY 2016,5500,0,0,1,0,1
2,020000005,CY 2020,5500,0,0,2,0,2
3,020000005,CY 2021,11000,0,1,1,0,2
4,020000012,CY 2012,1000,0,0,0,7,7


In [17]:
lcrtbl.head()

,Unnamed: 0,PWSID,Year,HB,M_R,PN_O,County,Jurisdiction,CN_ST,CN_ST_YR
0,0,020000012,2013,0,1,0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2013
1,1,020000012,2014,0,1,0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2014
2,2,020000012,2015,0,1,0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2015
3,3,020000012,2016,0,1,0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2016
4,4,020000012,2017,0,1,0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2017


In [18]:
GLPWSs_summ['Year'] = GLPWSs_summ['calendar_year'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

GLPWSs_counties = GLPWSs_summ.merge(lcrtbl, left_on=['pws_id','Year'], right_on=['PWSID','Year'],
          suffixes=('', '_lead'), how='left')

In [19]:
GLPWSs_counties.head(10)

,pws_id,calendar_year,population_served,HB,AHB,M_R,PN_O,Total,Year,Unnamed: 0,PWSID,HB_lead,M_R_lead,PN_O_lead,County,Jurisdiction,CN_ST,CN_ST_YR
0,020000005,CY 2015,5500,0,0,1,0,1,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,020000005,CY 2016,5500,0,0,1,0,1,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,020000005,CY 2020,5500,0,0,2,0,2,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,020000005,CY 2021,11000,0,1,1,0,2,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,020000012,CY 2012,1000,0,0,0,7,7,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,020000012,CY 2014,2000,0,2,1,0,3,2014,1.0,020000012,0.0,1.0,0.0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2014
6,020000012,CY 2015,2000,0,0,4,1,5,2015,2.0,020000012,0.0,1.0,0.0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2015
7,020000012,CY 2016,2000,0,0,1,2,3,2016,3.0,020000012,0.0,1.0,0.0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2016
8,020000012,CY 2018,2000,0,0,12,2,14,2018,5.0,020000012,0.0,1.0,0.0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2018
9,020000012,CY 2019,2000,0,0,9,4,13,2019,6.0,020000012,0.0,1.0,0.0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Onondaga Nation of New York_NY_2019


In [20]:
# subtract LCR violations from dataset for treatment dashboard
GLPWSs_counties = GLPWSs_counties.fillna(0)

In [21]:
GLPWSs_counties['HB'] = GLPWSs_counties['HB'] - GLPWSs_counties['HB_lead'] 
GLPWSs_counties['M_R'] = GLPWSs_counties['M_R'] - GLPWSs_counties['M_R_lead'] 
GLPWSs_counties['PN_O'] = GLPWSs_counties['PN_O'] - GLPWSs_counties['PN_O_lead'] 

In [22]:
# EPA doesn't seem to follow their own rules about how long a violation 'counts', so subtracting the LCR violations from the dashboard data results in a few negative values, I don't have a good solution for this currently

cols = ['HB', 'M_R', 'PN_O']
#GLPWSs_counties[GLPWSs_counties[cols] < 0] = 0
for col in cols:
    GLPWSs_counties.loc[GLPWSs_counties[col] < 0, col] = 0

In [23]:
GLPWSs_counties  = GLPWSs_counties.iloc[: , :8]


In [24]:
GLPWSs_counties.head(10)

,pws_id,calendar_year,population_served,HB,AHB,M_R,PN_O,Total
0,020000005,CY 2015,5500,0.0,0,1.0,0.0,1
1,020000005,CY 2016,5500,0.0,0,1.0,0.0,1
2,020000005,CY 2020,5500,0.0,0,2.0,0.0,2
3,020000005,CY 2021,11000,0.0,1,1.0,0.0,2
4,020000012,CY 2012,1000,0.0,0,0.0,7.0,7
5,020000012,CY 2014,2000,0.0,2,0.0,0.0,3
6,020000012,CY 2015,2000,0.0,0,3.0,1.0,5
7,020000012,CY 2016,2000,0.0,0,0.0,2.0,3
8,020000012,CY 2018,2000,0.0,0,11.0,2.0,14
9,020000012,CY 2019,2000,0.0,0,8.0,4.0,13


In [25]:
#update totals
GLPWSs_counties['Total'] = GLPWSs_counties['HB'] + GLPWSs_counties['AHB'] + GLPWSs_counties['M_R'] + GLPWSs_counties['PN_O']


In [26]:
GLPWSs_counties.head(10)

,pws_id,calendar_year,population_served,HB,AHB,M_R,PN_O,Total
0,020000005,CY 2015,5500,0.0,0,1.0,0.0,1.0
1,020000005,CY 2016,5500,0.0,0,1.0,0.0,1.0
2,020000005,CY 2020,5500,0.0,0,2.0,0.0,2.0
3,020000005,CY 2021,11000,0.0,1,1.0,0.0,2.0
4,020000012,CY 2012,1000,0.0,0,0.0,7.0,7.0
5,020000012,CY 2014,2000,0.0,2,0.0,0.0,2.0
6,020000012,CY 2015,2000,0.0,0,3.0,1.0,4.0
7,020000012,CY 2016,2000,0.0,0,0.0,2.0,2.0
8,020000012,CY 2018,2000,0.0,0,11.0,2.0,13.0
9,020000012,CY 2019,2000,0.0,0,8.0,4.0,12.0


In [27]:
# add county names to table from SW dataset
GLPWSs_counties.rename(columns = {'pws_id':'PWSID'}, inplace = True)

GLPWSs_counties = pd.merge(GLPWSs_counties,pwstbl[['PWSID','County', 'Jurisdiction']],on='PWSID', how='left')
# GLPWSs_counties.to_csv("GLPWSs_counties.csv") # optional data check
# POPULATION_SERVED_COUNT varies by year

In [28]:
cols = ['County', 'Jurisdiction']
#GLPWSs_counties['County'] = GLPWSs_counties['County'].fillna('Tribal Systems')
GLPWSs_counties['CN_ST'] = GLPWSs_counties[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

#flag tribal water systems
GLPWSs_counties['Tribal'] = 'N'
GLPWSs_counties.loc[GLPWSs_counties['Jurisdiction'].str.endswith('(Tribal)'), 'Tribal'] = 'Y'


#add tribe names for tribal systems
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '020000012', 'CN_ST'] = 'Onondaga Nation of New York_NY'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293201', 'CN_ST'] = 'Saginaw Chippewa Indian Tribe of Michigan_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293501', 'CN_ST'] = 'Sault Ste. Marie Tribe of Chippewa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055294101', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055294802', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055294803', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295508', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295509', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295510', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295601', 'CN_ST'] = 'Sokaogon Chippewa Community, Wisconsin_WI'

GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295703', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295704', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295705', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295706', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295802', 'CN_ST'] = 'Red Cliff Band of Lake Superior Chippewa Indians of Wisconsin_WI'

GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '020000005', 'CN_ST'] = 'St. Regis Band of Mohawk Indians of New York_NY'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293101', 'CN_ST'] = 'Bay Mills Indian Community_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293302', 'CN_ST'] = 'Keweenaw Bay Indian Community_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293304', 'CN_ST'] = 'Keweenaw Bay Indian Community_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293401', 'CN_ST'] = 'Lac Vieux Desert Band of Lake Superior Chippewa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293502', 'CN_ST'] = 'Sault Ste. Marie Tribe of Chippewa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293504', 'CN_ST'] = 'Sault Ste. Marie Tribe of Chippewa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293601', 'CN_ST'] = 'Grand Traverse Band of Ottawa and Chippewa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293603', 'CN_ST'] = 'Grand Traverse Band of Ottawa and Chippewa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293611', 'CN_ST'] = 'Hannahville Indian Community_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293702', 'CN_ST'] = 'Little River Band of Ottawa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293801', 'CN_ST'] = 'Little Traverse Bay Bands of Odawa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293802', 'CN_ST'] = 'Little Traverse Bay Bands of Odawa Indians_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055293901', 'CN_ST'] = 'Nottawaseppi Huron Band of Potawatomi_MI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055294801', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295003', 'CN_ST'] = 'Stockbridge Munsee Community_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295102', 'CN_ST'] = 'Bad River Band of Lake Superior Chippewa_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295103', 'CN_ST'] = 'Bad River Band of Lake Superior Chippewa_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295104', 'CN_ST'] = 'Bad River Band of Lake Superior Chippewa_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295201', 'CN_ST'] = 'Forest County Potawatomi Community_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295205', 'CN_ST'] = 'Forest County Potawatomi Community_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295504', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295505', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GLPWSs_counties.loc[GLPWSs_counties['PWSID'] == '055295507', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'




GLPWSs_counties['Year'] = GLPWSs_counties['calendar_year'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [29]:
GLPWSs_counties.head()
#GLPWSs_counties = GLPWSs_counties.drop(['calendar_year', '', axis=1)


,PWSID,calendar_year,population_served,HB,AHB,M_R,PN_O,Total,County,Jurisdiction,CN_ST,Tribal,Year
0,020000005,CY 2015,5500,0.0,0,1.0,0.0,1.0,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2015
1,020000005,CY 2016,5500,0.0,0,1.0,0.0,1.0,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2016
2,020000005,CY 2020,5500,0.0,0,2.0,0.0,2.0,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2020
3,020000005,CY 2021,11000,0.0,1,1.0,0.0,2.0,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2021
4,020000012,CY 2012,1000,0.0,0,0.0,7.0,7.0,Onondaga,EPA Region 2 (Tribal),Onondaga Nation of New York_NY,Y,2012


In [30]:
#calculate all numbers of violations, all numbers of systems with violations, and population affected by any violations for each county/year
df_cn = GLPWSs_counties.groupby(['CN_ST', 'calendar_year']).agg(n_sys_anyviol =('PWSID', 'nunique'), n_sys_hb=('HB', lambda x: (x > 0).sum()), 
    n_sys_AHB =('AHB', lambda x: (x > 0).sum()), n_sys_M_R =('M_R', lambda x: (x > 0).sum()), 
    n_sys_PN_O =('PN_O', lambda x: (x > 0).sum()),  n_viol_total = ('Total', 'sum'), 
    n_viol_HB = ('HB', 'sum'), n_viol_AHB = ('AHB', 'sum'), n_viol_M_R = ('M_R', 'sum'), 
    n_viol_PN_O = ('PN_O', 'sum'), pop_anyviol = ('population_served', 'sum')).reset_index()


In [31]:
# calculate population affected by each category of violation                                 
                                      
df_cn_hb = GLPWSs_counties[GLPWSs_counties['HB'].gt(0)].groupby(['CN_ST', 'calendar_year']).agg(pop_HB = ('population_served', 'sum')).reset_index()
df_cn_ahb = GLPWSs_counties[GLPWSs_counties['AHB'].gt(0)].groupby(['CN_ST', 'calendar_year']).agg(pop_AHB = ('population_served', 'sum')).reset_index()
df_cn_m_r = GLPWSs_counties[GLPWSs_counties['M_R'].gt(0)].groupby(['CN_ST', 'calendar_year']).agg(pop_M_R = ('population_served', 'sum')).reset_index()
df_cn_PN_O = GLPWSs_counties[GLPWSs_counties['PN_O'].gt(0)].groupby(['CN_ST', 'calendar_year']).agg(pop_PN_O = ('population_served', 'sum')).reset_index()

# mush all of the aggregates together
data_frames = [df_cn, df_cn_hb, df_cn_ahb, df_cn_m_r, df_cn_PN_O]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['CN_ST', 'calendar_year'],
                                            how='outer'), data_frames).fillna(0)

In [32]:
df_merged['Year'] = df_merged['calendar_year'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [33]:
tribes = ['Onondaga Nation of New York_NY', 'Saginaw Chippewa Indian Tribe of Michigan_MI', 'Sault Ste. Marie Tribe of Chippewa Indians of Michigan_MI', 'Minnesota Chippewa Tribe_MN', 'Menominee Indian Tribe of Wisconsin_WI', 'Sokaogon Chippewa Community, Wisconsin_WI',  'Oneida Tribe of Indians of Wisconsin_WI', 'Red Cliff Band of Lake Superior Chippewa Indians of Wisconsin_WI', 'St. Regis Band of Mohawk Indians of New York_NY', 'Bay Mills Indian Community_MI', 'Keweenaw Bay Indian Community_MI', 'Lac Vieux Desert Band of Lake Superior Chippewa Indians_MI', 'Sault Ste. Marie Tribe of Chippewa Indians_MI', 'Grand Traverse Band of Ottawa and Chippewa Indians_MI', 'Hannahville Indian Community_MI', 'Little River Band of Ottawa Indians_MI', 'Little Traverse Bay Bands of Odawa Indians_MI', 'Nottawaseppi Huron Band of Potawatomi_MI', 'Stockbridge Munsee Community_WI', 'Bad River Band of Lake Superior Chippewa_WI', 'Forest County Potawatomi Community_WI', 'Menominee Indian Tribe of Wisconsin_WI']

In [34]:
df_merged['Tribal'] = 'N'
df_merged.loc[df_merged['CN_ST'].isin(tribes), 'Tribal'] = 'Y'


In [35]:
df_merged.head()

,CN_ST,calendar_year,n_sys_anyviol,n_sys_hb,n_sys_AHB,n_sys_M_R,n_sys_PN_O,n_viol_total,n_viol_HB,n_viol_AHB,n_viol_M_R,n_viol_PN_O,pop_anyviol,pop_HB,pop_AHB,pop_M_R,pop_PN_O,Year,Tribal
0,Adams_IN,CY 2012,3,1,0,2,1,9.0,1.0,0,2.0,6.0,575,240.0,0.0,490.0,85.0,2012,N
1,Adams_IN,CY 2013,2,0,0,1,1,8.0,0.0,0,2.0,6.0,335,0.0,0.0,250.0,85.0,2013,N
2,Adams_IN,CY 2014,2,0,0,0,2,7.0,0.0,0,0.0,7.0,8735,0.0,0.0,0.0,8735.0,2014,N
3,Adams_IN,CY 2015,2,0,0,0,2,5.0,0.0,0,0.0,5.0,420,0.0,0.0,0.0,420.0,2015,N
4,Adams_IN,CY 2016,2,0,0,0,2,7.0,0.0,0,0.0,7.0,420,0.0,0.0,0.0,420.0,2016,N


In [36]:
## need num systems and total population served for each county/year, including systems with no violations

filenames = glob.glob(sfldr + "\*.xlsx")
print('File names:', filenames)

stbl = pd.DataFrame()

for file in filenames:
    df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
    stbl = stbl.append(df, ignore_index=True)
    

# fix column names
stbl.columns = [c.lower().replace(' ', '_') for c in stbl.columns]



File names: ['N:\\Projects\\Blue Accounting\\Issues\\Drinking Water\\Water treatment\\SDWA_dashboard_downloads_2023\\systems\\98f2331b-75e3-4121-8e57-b48e8b627448.xlsx', 'N:\\Projects\\Blue Accounting\\Issues\\Drinking Water\\Water treatment\\SDWA_dashboard_downloads_2023\\systems\\c0073757-fe75-4cac-91b2-f1e7a08240cb.xlsx']


C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\2317815694.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stbl = stbl.append(df, ignore_index=True)
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\2317815694.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stbl = stbl.append(df, ignore_index=True)


In [37]:
#repeat the same formatting done earlier for the treatment table
GL_all_PWSs = stbl[(stbl['pws_id'].isin(PWSs)) & (stbl['water_system_type'] == 'Community')]

GL_all_PWSs.rename(columns = {'pws_id':'PWSID'}, inplace = True)
GL_all_counties = pd.merge(GL_all_PWSs,pwstbl[['PWSID','County', 'Jurisdiction']],on='PWSID', how='left')

cols = ['County', 'Jurisdiction']
#GLPWSs_counties['County'] = GLPWSs_counties['County'].fillna('Tribal Systems')
GL_all_counties['CN_ST'] = GL_all_counties[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

GL_all_counties.loc[GL_all_counties['PWSID'] == '020000012', 'CN_ST'] = 'Onondaga Nation of New York_NY'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293201', 'CN_ST'] = 'Saginaw Chippewa Indian Tribe of Michigan_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293501', 'CN_ST'] = 'Sault Ste. Marie Tribe of Chippewa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055294101', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055294802', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055294803', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295508', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295509', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295510', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295601', 'CN_ST'] = 'Sokaogon Chippewa Community, Wisconsin_WI'

GL_all_counties.loc[GL_all_counties['PWSID'] == '055295703', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295704', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295705', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295706', 'CN_ST'] = 'Oneida Tribe of Indians of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295802', 'CN_ST'] = 'Red Cliff Band of Lake Superior Chippewa Indians of Wisconsin_WI'

GL_all_counties.loc[GL_all_counties['PWSID'] == '020000005', 'CN_ST'] = 'St. Regis Band of Mohawk Indians of New York_NY'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293101', 'CN_ST'] = 'Bay Mills Indian Community_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293302', 'CN_ST'] = 'Keweenaw Bay Indian Community_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293304', 'CN_ST'] = 'Keweenaw Bay Indian Community_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293401', 'CN_ST'] = 'Lac Vieux Desert Band of Lake Superior Chippewa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293502', 'CN_ST'] = 'Sault Ste. Marie Tribe of Chippewa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293504', 'CN_ST'] = 'Sault Ste. Marie Tribe of Chippewa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293601', 'CN_ST'] = 'Grand Traverse Band of Ottawa and Chippewa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293603', 'CN_ST'] = 'Grand Traverse Band of Ottawa and Chippewa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293611', 'CN_ST'] = 'Hannahville Indian Community_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293702', 'CN_ST'] = 'Little River Band of Ottawa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293801', 'CN_ST'] = 'Little Traverse Bay Bands of Odawa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293802', 'CN_ST'] = 'Little Traverse Bay Bands of Odawa Indians_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055293901', 'CN_ST'] = 'Nottawaseppi Huron Band of Potawatomi_MI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055294801', 'CN_ST'] = 'Minnesota Chippewa Tribe_MN'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295003', 'CN_ST'] = 'Stockbridge Munsee Community_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295102', 'CN_ST'] = 'Bad River Band of Lake Superior Chippewa_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295103', 'CN_ST'] = 'Bad River Band of Lake Superior Chippewa_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295104', 'CN_ST'] = 'Bad River Band of Lake Superior Chippewa_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295201', 'CN_ST'] = 'Forest County Potawatomi Community_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295205', 'CN_ST'] = 'Forest County Potawatomi Community_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295504', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295505', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'
GL_all_counties.loc[GL_all_counties['PWSID'] == '055295507', 'CN_ST'] = 'Menominee Indian Tribe of Wisconsin_WI'

GL_all_counties['Tribal'] = 'N'
GL_all_counties.loc[GL_all_counties['Jurisdiction'].str.endswith('(Tribal)'), 'Tribal'] = 'Y'

GL_all_counties['Year'] = GL_all_counties['calendar_year'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)


C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\1881067254.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GL_all_PWSs.rename(columns = {'pws_id':'PWSID'}, inplace = True)


In [38]:
#summarize total pop and num of systems for each county/tribe and year
years = list(range(startyear, (endyear+1))) # desired end year + 1

for year in years:
    print(year)
    df_year = GL_all_counties[(GL_all_counties['Year'] == year)]
    df_year_cn = df_year.groupby('CN_ST').agg(n_sys =('PWSID', 'count'), pop = ('population_served', 'sum')).reset_index()
    df_year_cn['Year'] = year
    if year == min(years):
        outf = df_year_cn
    else:
        outf = outf.append(df_year_cn)    

print(outf)

2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
            CN_ST  n_sys     pop  Year
0        Adams_IN      5    9905  2012
1        Adams_WI      3    1685  2012
2       Alcona_MI      5     689  2012
3        Alger_MI      5    4040  2012
4      Allegan_MI     47   33249  2012
..            ...    ...     ...   ...
224  Winnebago_WI     10  138194  2022
225       Wood_OH     21  107522  2022
226    Wyandot_OH      4   12047  2022
227    Wyoming_NY     23   21977  2022
228      Yates_NY     11   13684  2022

[2515 rows x 4 columns]


C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\1919391114.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outf = outf.append(df_year_cn)
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\1919391114.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outf = outf.append(df_year_cn)
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\1919391114.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outf = outf.append(df_year_cn)
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\1919391114.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outf = outf.append(df_year_cn)
C:\Users\agrimm\AppData\Local\Temp\ipykernel_19196\191939111

In [39]:
# final merge
final = pd.merge(outf,df_merged,on=['CN_ST', 'Year'], how='left').fillna(0)

# fix tribal flag for blank rows
final['Tribal'] = 'N'
final.loc[final['CN_ST'].isin(tribes), 'Tribal'] = 'Y'
final.head()


,CN_ST,n_sys,pop,Year,calendar_year,n_sys_anyviol,n_sys_hb,n_sys_AHB,n_sys_M_R,n_sys_PN_O,...,n_viol_HB,n_viol_AHB,n_viol_M_R,n_viol_PN_O,pop_anyviol,pop_HB,pop_AHB,pop_M_R,pop_PN_O,Tribal
0,Adams_IN,5,9905,2012,CY 2012,3.0,1.0,0.0,2.0,1.0,...,1.0,0.0,2.0,6.0,575.0,240.0,0.0,490.0,85.0,N
1,Adams_WI,3,1685,2012,CY 2012,3.0,1.0,0.0,1.0,1.0,...,1.0,0.0,2.0,2.0,3235.0,75.0,0.0,3100.0,3100.0,N
2,Alcona_MI,5,689,2012,CY 2012,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,2.0,1.0,110.0,0.0,0.0,110.0,110.0,N
3,Alger_MI,5,4040,2012,CY 2012,2.0,2.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,3083.0,3083.0,0.0,0.0,0.0,N
4,Allegan_MI,47,33249,2012,CY 2012,3.0,2.0,0.0,1.0,1.0,...,2.0,0.0,1.0,2.0,606.0,468.0,0.0,138.0,316.0,N


In [40]:
#all attributes for each county, year

final = final.drop('calendar_year', axis=1)
final.to_csv(r"N:\Projects\Blue Accounting\Issues\Drinking Water\Water treatment\GLPWSs_county_SDWIS_2012_2022.csv")

In [41]:
GL_all_counties.head()

,PWSID,pws_name,detailed_facility_report,calendar_year,primacy_agency_type,state/territory/tribe,city_served,population_served,water_system_type,system_size,...,enforcement_priority_flag,formal_enforcement_action_flag,informal_enforcement_action_flag,return_to_compliance_flag,enforcement_priority_rtc_flag,County,Jurisdiction,CN_ST,Tribal,Year
0,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2012,EPA,Unknown Tribe,-,5500,Community,Medium,...,No,No,No,No,No,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2012
1,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2013,EPA,Unknown Tribe,-,5500,Community,Medium,...,No,No,No,No,No,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2013
2,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2014,EPA,Unknown Tribe,-,5500,Community,Medium,...,No,No,No,No,No,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2014
3,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2015,EPA,Unknown Tribe,-,5500,Community,Medium,...,No,No,No,No,No,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2015
4,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,CY 2016,EPA,St. Regis Band of Mohawk Indians of New York,-,5500,Community,Medium,...,No,No,No,Yes,No,Franklin,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2016


In [42]:
#generate complete table of annual violations per system, including active systems with no violations
results = []
for year in years:
    print(year)
    GLPWSs_year = GL_all_counties[(GL_all_counties['Year'] == year)]
    
    systems_year = GLPWSs_year["PWSID"].unique() 
    
    for n in systems_year:
        #print(n)
        a = GLPWSs_counties.loc[(GLPWSs_counties['PWSID'] == n) & (GLPWSs_counties['Year'] == year)]
        name = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'pws_name'].iloc[0]
        dfr = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'detailed_facility_report'].iloc[0]
        if a.empty:
            pop = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'population_served'].iloc[0]
            CN_ST = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'CN_ST'].iloc[0]
            county = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'County'].iloc[0]
            jurisd = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'Jurisdiction'].iloc[0]
            tribe = GLPWSs_year.loc[GLPWSs_year['PWSID'] == n, 'Tribal'].iloc[0]
            out = [n, name, dfr, 'CY '+str(year), pop, 0, 0, 0, 0, 0, county, jurisd, CN_ST, tribe, year]
        else:
            out = a.values.flatten().tolist()
            out.insert(1, name)
            out.insert(2, dfr)

        #print(out)
        results.append(out)

results.sort()

df_final = pd.DataFrame(results, columns = ['PWSID', 'pws_name', 'detailed_facility_report', 'calendar_year', 'population_served', 'HB', 'AHB', 'M_R', 'PN_O', 'Total', 'County', 'Jurisdiction', 'CN_ST', 'Tribal', 'Year'])

2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [43]:
df_final = df_final.drop(['calendar_year', 'Total', 'County'], axis=1)

In [44]:
df_final.head()

,PWSID,pws_name,detailed_facility_report,population_served,HB,AHB,M_R,PN_O,Jurisdiction,CN_ST,Tribal,Year
0,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,5500,0.0,0,0.0,0.0,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2019
1,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,5500,0.0,0,2.0,0.0,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2020
2,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,11000,0.0,1,1.0,0.0,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2021
3,020000005,SAINT REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,5500,0.0,0,0.0,0.0,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2022
4,020000005,ST. REGIS MOHAWK TRIBE,https://echo.epa.gov/detailed-facility-report?...,5500,0.0,0,0.0,0.0,EPA Region 2 (Tribal),St. Regis Band of Mohawk Indians of New York_NY,Y,2012


In [45]:
#attributes for individual systems

df_final.to_csv(r"N:\Projects\Blue Accounting\Issues\Drinking Water\Water treatment\GLPWSs_systems_SDWIS_2012_2022.csv")